In [ ]:
import asyncio
import os
from collections.abc import Sequence
from typing import Any

from dotenv import load_dotenv
from agent_framework import AgentSession, BaseHistoryProvider, Message
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity.aio import AzureCliCredential

In [ ]:
load_dotenv(override=True)

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

In [ ]:
class CustomHistoryProvider(BaseHistoryProvider):
    def __init__(self) -> None:
        super().__init__("custom-history")
        self._storage: dict[str, list[Message]] = {}

    async def get_messages(
        self, session_id: str | None, *, state: dict[str, Any] | None = None, **kwargs: Any
    ) -> list[Message]:
        key = session_id or "default"
        return list(self._storage.get(key, []))

    async def save_messages(
        self,
        session_id: str | None,
        messages: Sequence[Message],
        *,
        state: dict[str, Any] | None = None,
        **kwargs: Any,
    ) -> None:
        key = session_id or "default"

        if key not in self._storage:
            self._storage[key] = []
        self._storage[key].extend(messages)

In [ ]:

credential = AzureCliCredential()
client = AzureOpenAIResponsesClient(
    project_endpoint=project_endpoint,
    deployment_name=model,
    credential=credential,
)

agent = client.as_agent(
    name="CustomBot",
    instructions="You are a helpful assistant that remembers our conversation.",
    context_providers=[CustomHistoryProvider()],
)

session = agent.create_session()

In [ ]:
query = "Hello! My name is Alice and I love pizza."

print(f"User: {query}")
print(f"Agent: {await agent.run(query, session=session)}\n")

In [ ]:
serialized_session = session.to_dict()

print(f"Serialized session: {serialized_session}\n")

resumed_session = AgentSession.from_dict(serialized_session)

In [ ]:
query = "What do you remember about me?"

print(f"User: {query}")
print(f"Agent: {await agent.run(query, session=resumed_session)}\n")